## selection 개수 확인

In [0]:
sel_vc = train.loc[train['event']==1,'event_contents'].value_counts()

In [0]:
sel_vc[:20]

[]                              811772
['OrbitalCommand [3100001]']     75165
['Nexus [3100001]']              55956
['OrbitalCommand [3440001]']     52412
['Lair [3100001]']               47356
['Nexus [3440001]']              38817
['Lair [3440001]']               29997
['Hive [3100001]']               28367
['SCV [37C0001]']                27178
['OrbitalCommand [3080001]']     23561
['OrbitalCommand [3540001]']     22337
['Hive [3440001]']               21290
['OrbitalCommand [3380001]']     19858
['OrbitalCommand [3000001]']     19846
['Barracks [39C0001]']           19781
['Hatchery [3100001]']           19022
['Nexus [3080001]']              17860
['SCV [38C0001]']                16964
['Barracks [3A40001]']           16768
['SCV [3740001]']                16366
Name: event_contents, dtype: int64

- selection의 event_contents엔 아무의미 없는 빈 리스트가 81만개 존재한다.

## test selection event_contents 개수 확인

In [0]:
t_sel_vc = test.loc[test['event']==1,'event_contents'].value_counts()

In [0]:
t_sel_vc[:20]

[]                              345691
['OrbitalCommand [3100001]']     32721
['Nexus [3100001]']              24151
['OrbitalCommand [3440001]']     20455
['Lair [3100001]']               19155
['Nexus [3440001]']              17548
['Hive [3100001]']               13413
['Lair [3440001]']               13287
['SCV [37C0001]']                11100
['OrbitalCommand [3080001]']     10852
['OrbitalCommand [3540001]']      9911
['Hive [3440001]']                9263
['OrbitalCommand [3000001]']      8889
['Barracks [39C0001]']            8525
['Hatchery [3100001]']            8425
['OrbitalCommand [3380001]']      8424
['Nexus [3080001]']               7784
['SCV [3740001]']                 7217
['SCV [38C0001]']                 7117
['SCV [3580001]']                 6947
Name: event_contents, dtype: int64

- test 데이터의 selection의 event_contents도 마찬가지로 의미없는 빈 리스트 존재

# preprocessing

## selection null 제거

In [0]:
# train_sel= train.loc[train['event']==1]
test_sel = test.loc[test['event']==1]

In [0]:
rm_null_sel= train_sel[train_sel['event_contents']!='[]']
rm_null_sel

,game_id,winner,time,player,species,event,event_contents
2,0.0,1.0,0,0.0,0.0,1.0,['OrbitalCommand [3080001]']
28,0.0,1.0,0,0.0,0.0,1.0,['OrbitalCommand [3080001]']
57,0.0,1.0,0,1.0,0.0,1.0,['OrbitalCommand [33C0001]']
68,0.0,1.0,0,0.0,0.0,1.0,['SCV [3280001]']
102,0.0,1.0,1,0.0,0.0,1.0,['CreepOnlyBlocker4x4 [1B00001]']
...,...,...,...,...,...,...,...
67091719,38871.0,0.0,8,0.0,2.0,1.0,['Egg [5700004]']
67091720,38871.0,0.0,8,0.0,2.0,1.0,['Egg [5700004]']
67091723,38871.0,0.0,8,0.0,2.0,1.0,['Drone [4300006]']
67091748,38871.0,0.0,8,0.0,2.0,1.0,['Egg [47C0004]']


In [0]:
t_rm_null_sel = test_sel[test_sel['event_contents']!='[]']

In [0]:
t_rm_null_sel

,game_id,time,player,species,event,event_contents
2,38872.0,0,1.0,1.0,1.0,['Nexus [3100001]']
31,38872.0,0,0.0,1.0,1.0,['Probe [2F80001]']
65,38872.0,0,1.0,1.0,1.0,['Probe [32C0001]']
86,38872.0,0,0.0,1.0,1.0,['Probe [2F40001]']
97,38872.0,0,0.0,1.0,1.0,['Nexus [2DC0001]']
...,...,...,...,...,...,...
28714788,55658.0,4,0.0,2.0,1.0,['Egg [4440001]']
28714803,55658.0,4,1.0,0.0,1.0,"['Reaper [4000001]', 'Reaper [4100002]', 'Reap..."
28714812,55658.0,4,0.0,2.0,1.0,"['Zergling [4140003]', 'Zergling [4380003]']"
28714813,55658.0,4,0.0,2.0,1.0,"['Zergling [4140003]', 'Zergling [4380003]']"


In [0]:
len(t_rm_null_sel[(t_rm_null_sel.player==0 )&(t_rm_null_sel.game_id==38873 )])

107

In [0]:
def prepro_selection(data,i):
    p0_selection =[]
    p1_selection = []
    # for i in b['game_id'].unique():
    p0_selection.append(len(data.loc[(data.player==0) &(data.game_id==i)]))
    p1_selection.append(len(data.loc[(data.player==1) &(data.game_id==i)]))

    
    return p0_selection,p1_selection


In [0]:
import threading
import concurrent.futures
import time
p0_sel,p1_sel = [],[]
t_start = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    t= [executor.submit(prepro_selection,t_rm_null_sel,i) for i in t_rm_null_sel['game_id'].unique()]


for i in t:
    a,b = i.result()
    p0_sel +=a
    p1_sel +=b

x_t['P0_p_selection'] = p0_sel
x_t['P1_p_selection'] = p1_sel
x_t['delta_p_selection'] = x_t['P0_p_selection'] - x_t['P1_p_selection']
x_t.to_csv('x_test.csv')

In [0]:
p0_sel

[48, 107, 41, 114, 25, 55, 14, 106, 252, 304]

In [0]:
x_train = pd.read_csv('xy_train2.csv')
print(x_train.shape)
x_train.head()

(38872, 32)


,Unnamed: 0,Unnamed: 0.1,game_id,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,P1_species,P1_Camera,P1_Selection,P1_Ability,P1_Right Click,P1_SetControlGroup,P1_GetControlGroup,P1_AddToControlGroup,P1_ControlGroup,delta_Camera,delta_Selection,delta_Ability,delta_Right Click,delta_SetControlGroup,delta_GetControlGroup,delta_AddToControlGroup,delta_ControlGroup,winner,p0_wm,p1_wm
0,0,0,0.0,0.0,444.0,50.0,34.0,35.0,3.0,24.0,2.0,0.0,0.0,425.0,57.0,34.0,28.0,1.0,3.0,0.0,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,1.0,9,6
1,1,1,1.0,1.0,627.0,186.0,77.0,160.0,10.0,162.0,1.0,0.0,0.0,858.0,116.0,67.0,131.0,8.0,31.0,0.0,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1.0,22,17
2,2,2,2.0,1.0,413.0,90.0,69.0,160.0,14.0,99.0,6.0,0.0,2.0,725.0,232.0,85.0,204.0,9.0,109.0,5.0,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,0.0,19,23
3,3,3,3.0,0.0,713.0,180.0,82.0,276.0,6.0,132.0,0.0,0.0,1.0,388.0,148.0,89.0,268.0,19.0,710.0,0.0,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,0.0,7,24
4,4,4,4.0,0.0,430.0,67.0,57.0,177.0,10.0,224.0,1.0,0.0,2.0,272.0,126.0,36.0,106.0,8.0,99.0,4.0,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0.0,14,13


In [0]:
x_t = pd.read_csv('x_test2.csv')
print(x_t.shape)
x_t.head()

(16787, 32)


,Unnamed: 0,Unnamed: 0.1,game_id,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,P1_species,P1_Camera,P1_Selection,P1_Ability,P1_Right Click,P1_SetControlGroup,P1_GetControlGroup,P1_AddToControlGroup,P1_ControlGroup,delta_Camera,delta_Selection,delta_Ability,delta_Right Click,delta_SetControlGroup,delta_GetControlGroup,delta_AddToControlGroup,delta_ControlGroup,p0_wm,p1_wm,delta_wm
0,0,0,38872.0,0.0,232.0,52.0,31.0,1.0,5.0,241.0,9.0,0.0,0.0,467.0,41.0,38.0,4.0,0.0,122.0,43.0,0.0,-235.0,11.0,-7.0,-3.0,5.0,119.0,-34.0,0.0,12,11,1
1,1,1,38873.0,1.0,200.0,114.0,34.0,0.0,0.0,51.0,18.0,4.0,0.0,146.0,19.0,28.0,1.0,5.0,68.0,122.0,0.0,54.0,95.0,6.0,-1.0,-5.0,-17.0,-104.0,4.0,3,7,-4
2,2,2,38874.0,2.0,245.0,42.0,33.0,0.0,7.0,79.0,63.0,0.0,2.0,220.0,36.0,29.0,0.0,5.0,108.0,426.0,0.0,25.0,6.0,4.0,0.0,2.0,-29.0,-363.0,0.0,10,4,6
3,3,3,38875.0,2.0,515.0,123.0,95.0,8.0,9.0,218.0,255.0,0.0,0.0,357.0,99.0,58.0,1.0,2.0,322.0,24.0,0.0,158.0,24.0,37.0,7.0,7.0,-104.0,231.0,0.0,10,12,-2
4,4,4,38876.0,1.0,129.0,25.0,7.0,2.0,2.0,30.0,387.0,0.0,1.0,161.0,35.0,8.0,0.0,1.0,7.0,1.0,0.0,-32.0,-10.0,-1.0,2.0,1.0,23.0,386.0,0.0,1,5,-4


In [0]:
x_t

,Unnamed: 0,Unnamed: 0.1,game_id,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,P1_species,P1_Camera,P1_Selection,P1_Ability,P1_Right Click,P1_SetControlGroup,P1_GetControlGroup,P1_AddToControlGroup,P1_ControlGroup,delta_Camera,delta_Selection,delta_Ability,delta_Right Click,delta_SetControlGroup,delta_GetControlGroup,delta_AddToControlGroup,delta_ControlGroup,p0_wm,p1_wm,delta_wm,P0_p_selection,P1_p_selection,delta_p_selection
0,0,0,38872.0,0.0,232.0,52.0,31.0,1.0,5.0,241.0,9.0,0.0,0.0,467.0,41.0,38.0,4.0,0.0,122.0,43.0,0.0,-235.0,11.0,-7.0,-3.0,5.0,119.0,-34.0,0.0,12,11,1,48,38,10
1,1,1,38873.0,1.0,200.0,114.0,34.0,0.0,0.0,51.0,18.0,4.0,0.0,146.0,19.0,28.0,1.0,5.0,68.0,122.0,0.0,54.0,95.0,6.0,-1.0,-5.0,-17.0,-104.0,4.0,3,7,-4,107,18,89
2,2,2,38874.0,2.0,245.0,42.0,33.0,0.0,7.0,79.0,63.0,0.0,2.0,220.0,36.0,29.0,0.0,5.0,108.0,426.0,0.0,25.0,6.0,4.0,0.0,2.0,-29.0,-363.0,0.0,10,4,6,41,34,7
3,3,3,38875.0,2.0,515.0,123.0,95.0,8.0,9.0,218.0,255.0,0.0,0.0,357.0,99.0,58.0,1.0,2.0,322.0,24.0,0.0,158.0,24.0,37.0,7.0,7.0,-104.0,231.0,0.0,10,12,-2,114,88,26
4,4,4,38876.0,1.0,129.0,25.0,7.0,2.0,2.0,30.0,387.0,0.0,1.0,161.0,35.0,8.0,0.0,1.0,7.0,1.0,0.0,-32.0,-10.0,-1.0,2.0,1.0,23.0,386.0,0.0,1,5,-4,25,35,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16782,16782,16782,55654.0,1.0,258.0,186.0,37.0,2.0,6.0,136.0,693.0,0.0,2.0,318.0,108.0,53.0,15.0,14.0,149.0,515.0,0.0,-60.0,78.0,-16.0,-13.0,-8.0,-13.0,178.0,0.0,2,8,-6,162,104,58
16783,16783,16783,55655.0,1.0,810.0,317.0,62.0,0.0,7.0,158.0,217.0,0.0,2.0,638.0,103.0,75.0,2.0,0.0,174.0,50.0,0.0,172.0,214.0,-13.0,-2.0,7.0,-16.0,167.0,0.0,6,4,2,310,92,218
16784,16784,16784,55656.0,0.0,330.0,67.0,14.0,0.0,1.0,106.0,0.0,0.0,0.0,328.0,52.0,36.0,0.0,0.0,251.0,28.0,2.0,2.0,15.0,-22.0,0.0,1.0,-145.0,-28.0,-2.0,8,6,2,61,46,15
16785,16785,16785,55657.0,2.0,20.0,5.0,7.0,0.0,2.0,8.0,0.0,0.0,2.0,65.0,23.0,5.0,0.0,1.0,16.0,105.0,0.0,-45.0,-18.0,2.0,0.0,1.0,-8.0,-105.0,0.0,3,3,0,5,23,-18


In [0]:
a= train[train['event']==1]

In [0]:
a.loc[(a['game_id']==0)& (a['player']==1),'event_contents'].str.contains('36C0001').sum()

2

In [0]:
a[a['game_id']==1]

,game_id,winner,time,player,species,event,event_contents
1142,1.0,1.0,0,1.0,0.0,1.0,['OrbitalCommand [3340001]']
1143,1.0,1.0,0,0.0,1.0,1.0,['Nexus [3000001]']
1195,1.0,1.0,0,1.0,0.0,1.0,['SCV [36C0001]']
1210,1.0,1.0,0,0.0,1.0,1.0,['Probe [3100001]']
1235,1.0,1.0,0,0.0,1.0,1.0,"['Probe [3040001]', 'Probe [30C0001]', 'Probe ..."
...,...,...,...,...,...,...,...
3550,1.0,1.0,9,0.0,1.0,1.0,['TwilightCouncil [5800001]']
3554,1.0,1.0,9,0.0,1.0,1.0,['RoboticsFacility [4000001]']
3563,1.0,1.0,9,0.0,1.0,1.0,['Probe [3CC0001]']
3571,1.0,1.0,9,1.0,0.0,1.0,['Marine [4300001]']
